In [3]:
#pip install pandas
#pip install numpy
#pip install -U scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Counter proporciona un diccionario donde cada elemento distinto se almacena 
# como claves con un valor asociado igual a su cuenta
from collections import Counter
 

import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
 
 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [4]:

# Cargar los datos y separar variables y etiqueta
df = pd.read_csv('datos/datos10.csv')
print('####################### INFO sobre el dataset ########################')
print(df.info())
print('####################### los cinco primeros valores del dataset########################')
print(df.head())
# Df head devuelve los n primeros del dataframe , útil para ver que tenemos los tipos de datos esperados.

X = df.drop('clase_objetivo', axis = 1)
y = df['clase_objetivo']

# Dividir el conjunto de datos 

################################################################################################################ está puesto el parámetro de stratify a YES ####################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir las variables que requieren preprocesamiento
numeric_features = ['variable1', 'variable2']
categorical_features = ['variable_categorica']




################################################################################################################ PREPROCESAMOS CON UN PIPELINE 
# Crear un pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),#gestión de elementos vacíos
    ('scaler', StandardScaler())# Escalamos las variables numéricas
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),#gestión de elementos vacíos
    ('encoder', OneHotEncoder(handle_unknown='ignore')) #Convertimos las variables categóricas a una traducción numérica
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Incluir el clasificador en el pipeline
model = KNeighborsClassifier()

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Definir el grid con los parámetros - los parámetros son en función de lo que estés usando. (vecinos, árboles..)- fijate en los parámetros que hay disponibles a la hora de crear el clasificador
param_grid = {
    'classifier__n_neighbors': [3, 7, 11],#Vamos a probar con esos números como k en vecinos.
    'classifier__p': [1, 2] # p=1 es usar distancia Manhattan , p=2 distancia euclídea. SI NO PUSIÉSEMOS NADA USARÍA LA DISTANCIA MINKOWSKI
}

################################################################################################################ PREPROCESAMOS CON UN PIPELINE 

################################################################################################################ BÚSQUEDA DE MEJORES HIPERPARÁMETROS.

# Crear un obejeto GridSearchCV - Aqui no se  mostrarán las matrices, ya que estamos sacando los datos enfocado a la accuracy
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Entrenar el grid en el conjunto de datos de entrenamiento
grid_search.fit(X_train, y_train)
print(grid_search)
# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# Acceder al mejor modelo del grid_search
best_model = grid_search.best_estimator_
print(best_model) # esto es un pipeline incluye el preprocesado

# Utilizar el mejor modelo para hacer las precciones en el conjunto de test
y_pred = best_model.predict(X_test)

# Evaluar la accuracy en el conjunto de test
accuracy_test = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_test)

####################### INFO sobre el dataset ########################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   variable1            90 non-null     float64
 1   variable2            88 non-null     float64
 2   variable_categorica  100 non-null    object 
 3   clase_objetivo       100 non-null    object 
dtypes: float64(2), object(2)
memory usage: 3.3+ KB
None
####################### los cinco primeros valores del dataset########################
   variable1  variable2 variable_categorica clase_objetivo
0   2.084605  22.502084                   A         Clase3
1   4.816811  42.560540                   C         Clase1
2   4.205380  18.120843                   B         Clase1
3   8.591820        NaN                   C         Clase2
4   1.711616  43.609472                   C         Clase2
GridSearchCV(cv=5,
         